# TP afin de voir les different partie de notre presentation

### Travail Pratique : Détection du Cancer du Sein

### Objectif
### L'objectif de ce TP est de vous familiariser avec les techniques de traitement d'images, y compris le chargement des données, le prétraitement, la segmentation, le filtrage et la modélisation, et de déployer votre modèle dans une application Streamlit.

### Dataset
##### Utilisez le Wisconsin Breast Cancer Dataset disponible à l'adresse suivante : Breast Cancer Wisconsin Dataset.(https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data)

# Étapes du TP
### 1. Chargement et Préparation des Données
Instructions :
Téléchargez le dataset et chargez les images.
Redimensionnez les images à une taille standard (par exemple, 300x300 pixels).
Divisez les données en ensembles d'entraînement (150 images) et de test (50 images).

In [ ]:
import os
import cv2
import numpy as np

# Configuration des chemins et paramètres
path_train = 'Training/'   # Dossier d'images d'entraînement
path_test = 'Testing/'     # Dossier d'images de test
img_size = 300             # Taille de l'image après redimensionnement

# Initialisation des listes pour stocker les images et labels
train_img = []
train_labels = []
test_img = []
test_labels = []

# Chargement des images d'entraînement (150 images)
for i in os.listdir(path_train):
    for j in os.listdir(os.path.join(path_train, i)):
        if len(train_img) < 150:  # Limiter à 150 images pour l'entraînement
            img = cv2.resize(cv2.imread(os.path.join(path_train, i, j)), (img_size, img_size))
            train_img.append(img)
            train_labels.append(i)
        else:
            break

# Chargement des images de test (50 images)
for i in os.listdir(path_test):
    for j in os.listdir(os.path.join(path_test, i)):
        if len(test_img) < 50:  # Limiter à 50 images pour le test
            img = cv2.resize(cv2.imread(os.path.join(path_test, i, j)), (img_size, img_size))
            test_img.append(img)
            test_labels.append(i)
        else:
            break

# Conversion des listes en arrays NumPy
train_img = np.array(train_img)
train_labels = np.array(train_labels)
test_img = np.array(test_img)
test_labels = np.array(test_labels)

print(f"Nombre d'images d'entraînement : {len(train_img)}")
print(f"Nombre d'images de test : {len(test_img)}")


## 2. Filtrage d'Images
Instructions :
Appliquez différents filtres pour améliorer la qualité des images avant la segmentation.
Utilisez un filtre gaussien pour réduire le bruit et un filtre de Sobel pour détecter les contours.

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Fonction pour appliquer des filtres
def apply_filters(images):
    filtered_images = []
    for img in images:
        # Filtre gaussien pour réduire le bruit
        blurred = cv2.GaussianBlur(img, (5, 5), 0)
        # Filtre de Sobel pour détecter les contours
        sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=5)
        sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=5)
        sobel_combined = cv2.magnitude(sobel_x, sobel_y)
        filtered_images.append(sobel_combined)
    return np.array(filtered_images)

# Appliquer les filtres aux images d'entraînement
filtered_train_images = apply_filters(train_img)

# Affichage des résultats
plt.figure(figsize=(15, 10))
for i in range(5):
    plt.subplot(3, 5, i + 1)
    plt.imshow(cv2.cvtColor(train_img[i], cv2.COLOR_BGR2RGB))
    plt.title('Image Originale')
    plt.axis('off')

    plt.subplot(3, 5, i + 6)
    plt.imshow(filtered_train_images[i], cmap='gray')
    plt.title('Image Filtrée')
    plt.axis('off')

plt.show()


## 3. Segmentation d'Images
Instructions :
Appliquez une méthode de segmentation comme K-Means ou seuillage adaptatif sur les images filtrées.

In [ ]:
from sklearn.cluster import KMeans

def segment_images(images):
    segmented_images = []
    for img in images:
        img_flat = img.flatten().reshape(-1, 1)  # Aplatir l'image pour K-Means
        kmeans = KMeans(n_clusters=2)  # Deux clusters : tumeur vs non-tumeur
        labels = kmeans.fit_predict(img_flat)
        segmented_image = labels.reshape(img.shape).astype(np.uint8) * 255  # Convertir en image binaire
        segmented_images.append(segmented_image)
    return np.array(segmented_images)

# Appliquer la segmentation aux images filtrées
segmented_train_images = segment_images(filtered_train_images)

# Affichage des résultats de segmentation
plt.figure(figsize=(15, 10))
for i in range(5):
    plt.subplot(3, 5, i + 1)
    plt.imshow(cv2.cvtColor(filtered_train_images[i], cv2.COLOR_BGR2RGB))
    plt.title('Image Filtrée')
    plt.axis('off')

    plt.subplot(3, 5, i + 6)
    plt.imshow(segmented_train_images[i], cmap='gray')
    plt.title('Image Segmentée')
    plt.axis('off')

plt.show()


### 4. Modélisation (Fine-Tuning du Modèle Pré-entrainé)
Instructions :
Utilisez un modèle pré-entraîné comme VGG16 ou ResNet50.
Appliquez un fine-tuning sur votre ensemble d'entraînement.

In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
from datasets import Dataset

# Prétraitement des images pour le modèle (utiliser les images segmentées si nécessaire)
def preprocess_images(images):
    return [cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_GRAY2RGB) for img in images]

train_dataset_dict = {
    'image': preprocess_images(segmented_train_images),
    'label': train_labels.tolist()
}

test_dataset_dict = {
    'image': preprocess_images(test_img),
    'label': test_labels.tolist()
}

model_name = "google/vit-base-patch16-224-in21k"  
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(set(train_labels)))

train_dataset = Dataset.from_dict(train_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

def preprocess_function(examples):
    return feature_extractor(images=examples['image'], return_tensors="pt")

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

model.save_pretrained('./fine_tuned_model')
feature_extractor.save_pretrained('./fine_tuned_model')

print("Modèle sauvegardé avec succès !")


### 5. Évaluation du Modèle
Instructions :
Évaluez votre modèle sur l'ensemble de test et affichez les résultats.

In [ ]:
eval_results = trainer.evaluate()
print("Résultats de l'évaluation :", eval_results)


### 6. Application Streamlit pour la Prédiction
Instructions :
Créez une interface utilisateur simple où les utilisateurs peuvent télécharger une image et obtenir une prédiction.

In [ ]:
import streamlit as st
from PIL import Image

model = ViTForImageClassification.from_pretrained('./fine_tuned_model')
feature_extractor = ViTFeatureExtractor.from_pretrained('./fine_tuned_model')

st.title("Détection de Tumeur dans les Images Médicales")

uploaded_file = st.file_uploader("Choisissez une image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("L") # Convertir en niveaux de gris si nécessaire.
    
    # Appliquer le même prétraitement que précédemment (filtrage et segmentation si nécessaire).
    
    inputs = feature_extractor(images=image.convert("RGB"), return_tensors="pt")
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class = logits.argmax(-1).item()
    
    if predicted_class == 1:
        st.write("La tumeur a été détectée.")
    else:
        st.write("Aucune tumeur détectée.")
